In [3]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv

In [34]:
import pandas as pd

In [38]:
df = pd.read_csv('lda_lotte.csv',sep='\t',header=None)

In [44]:
df= df[[0]]

In [46]:
df.columns = ['document']

In [42]:
df_1 = pd.read_excel('/Users/a1101809/Downloads/lda_lotte_1.xlsx')

In [48]:
df_lotte = pd.concat([df,df_1])

### emart

In [53]:
df_emart = pd.read_csv('lda_emart.csv',sep='\t')

In [59]:
!pip install mecab-python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1233 sha256=528be294401d59ffaa4423de24bcb417cd32cee9e39bc8445b1eb5f6df0f510e
  Stored in directory: /Users/a1101809/Library/Caches/pip/wheels/51/ff/4a/9ef8960a821d12b4daabdfcbe5ebf06b5c70ad64fcfc23982b
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      /opt/homebrew/Caskroom/miniforge/base/envs/jupyter/lib/python3.8/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.0-arm64-cpython-

In [58]:
mecab= Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:

def clean_text(text):
    """
    한글, 영문, 숫자만 남기고 제거한다.
    :param text:
    :return:
    """
    text = text.replace(".", " ").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text


def get_nouns(tokenizer, sentence):
    """
    단어의 길이가 2이상인 일반명사(NNG), 고유명사(NNP), 외국어(SL)만을 반환한다.
    :param tokenizer:
    :param sentence:
    :return:
    """
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1]
    return nouns


def tokenize(df):
    tokenizer = Mecab(dicpath='C:/mecab/mecab-ko-dic')
    processed_data = []
    for sent in tqdm(df['description']):
        sentence = clean_text(sent.replace('\n', '').strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

def save_processed_data(processed_data):
    """
    토큰 분리한 데이터를 csv로 저장
    :param processed_data:
    :return:
    """
    with open('./data/tokenized_data.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)


if __name__ == '__main__':
    # 유공자 공훈조서 데이터를 읽어들인다.
    df = pd.read_csv('./data/historical_records.csv')
    processed_data = tokenize(df)   # description 부분을 토크나이징 한다.

    # 토큰 분리한 데이터를 저장
    save_processed_data(processed_data)